<a href="https://colab.research.google.com/github/Monsoon886-cyb/ML-Codes/blob/main/Anime_Recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd
import time

In [ ]:
def fetch_anime_data(limit):
  anime_list = []
  page = 1

  while len(anime_list) < limit:
    response = requests.get(f"https://api.jikan.moe/v4/top/anime?page={page}")

    if response.status_code == 200:
      data = response.json()['data']
      anime_list.extend(data)  # Use extend to add items to the list
    else:
      print(f"Error fetching data from page {page}: Status code {response.status_code}")


    page += 1
    time.sleep(1)
  return anime_list


In [ ]:
anime =  fetch_anime_data(10000)

In [ ]:
anime = pd.DataFrame(anime)
anime.to_csv("anime_data.csv", index=False)

In [ ]:
anime_df = pd.read_csv('/content/anime_data (1).csv')

In [ ]:
anime_df = anime_df[['mal_id','title','title_english','title_synonyms','studios','genres','explicit_genres','themes','demographics']]


In [ ]:
anime_df.iloc[1].demographics

"[{'mal_id': 27, 'type': 'anime', 'name': 'Shounen', 'url': 'https://myanimelist.net/anime/genre/27/Shounen'}]"

In [ ]:
import ast

In [ ]:
anime_df['title_synonyms']=anime_df['title_synonyms'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else [])


In [ ]:
def filter(obj):
  L=[]
  if pd.notna(obj):
    obj = ast.literal_eval(obj)
  else:
    []
  if len(obj)>0:
    for i in obj:
      L.append(i['name'])
  return L

In [ ]:
anime_df['studios'] = anime_df['studios'].apply(filter)
anime_df['genres'] = anime_df['genres'].apply(filter)
anime_df['explicit_genres'] = anime_df['explicit_genres'].apply(filter)
anime_df['themes'] = anime_df['themes'].apply(filter)
anime_df['demographics'] = anime_df['demographics'].apply(filter)

In [ ]:
def imp_titles(x, y):
    for i in range(len(anime_df)):
        if pd.isna(anime_df.at[i, x]):
            anime_df.at[i, x] = anime_df.at[i, y]
    return anime_df[x]

In [ ]:
anime_df['title1'] = imp_titles('title_english','title')

In [ ]:
anime_df

,mal_id,title,title_english,title_synonyms,studios,genres,explicit_genres,themes,demographics,title1
0,52991,Sousou no Frieren,Frieren: Beyond Journey's End,"[Frieren at the Funeral, Frieren The Slayer]",[Madhouse],"[Adventure, Drama, Fantasy]",[],[],[Shounen],Frieren: Beyond Journey's End
1,5114,Fullmetal Alchemist: Brotherhood,Fullmetal Alchemist: Brotherhood,[Hagane no Renkinjutsushi: Fullmetal Alchemist...,[Bones],"[Action, Adventure, Drama, Fantasy]",[],[Military],[Shounen],Fullmetal Alchemist: Brotherhood
2,9253,Steins;Gate,Steins;Gate,[],[White Fox],"[Drama, Sci-Fi, Suspense]",[],"[Psychological, Time Travel]",[],Steins;Gate
3,60022,One Piece Fan Letter,One Piece Fan Letter,[],[Toei Animation],"[Action, Adventure, Fantasy]",[],[],[Shounen],One Piece Fan Letter
4,38524,Shingeki no Kyojin Season 3 Part 2,Attack on Titan Season 3 Part 2,[],[Wit Studio],"[Action, Drama, Suspense]",[],"[Gore, Military, Survival]",[Shounen],Attack on Titan Season 3 Part 2
...,...,...,...,...,...,...,...,...,...,...
9995,3011,Yotsunoha,Yotsunoha,[],[HAL Film Maker],[Romance],[],[School],[],Yotsunoha
9996,49014,Daisan no Shinzou,The Third Heart,[],[Studio Gohan],[],[],[Music],[],The Third Heart
9997,835,Sister Princess: Re Pure,Sister Princess: Re Pure,[Sister Princess Repure],[Zexcs],"[Drama, Romance]",[],[Harem],[],Sister Princess: Re Pure
9998,2791,Makasete Iruka!,Grrl Power!,[],[CoMix Wave Films],"[Action, Award Winning, Comedy]",[],[School],[],Grrl Power!


In [ ]:
def split_sen(obj):
  results = []
  for s in obj:
    results += s.split(" ")
  return results

In [ ]:
anime_df['title_synonyms']=anime_df['title_synonyms'].apply(split_sen)

In [ ]:
anime_df['themes']=anime_df['themes'].apply(lambda x : [i.replace(" ","") for i in x])

In [ ]:
anime_df['studios']=anime_df['studios'].apply(lambda x : [i.replace(" ","") for i in x])

In [ ]:
anime_df['tags'] = anime_df['title_synonyms']+anime_df['themes']+anime_df['studios']+anime_df['genres']+anime_df['explicit_genres']+anime_df['demographics']

In [ ]:
anime_df = anime_df[['mal_id','title1','tags']]

In [ ]:
anime_df['tags'] = anime_df['tags'].apply(lambda x : " ".join(x))

<ipython-input-24-8cc391e44aaf>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  anime_df['tags'] = anime_df['tags'].apply(lambda x : " ".join(x))


In [ ]:
anime_df['tags']=anime_df['tags'].apply(lambda x: x.lower())

<ipython-input-25-01bc27cbf00a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  anime_df['tags']=anime_df['tags'].apply(lambda x: x.lower())


In [ ]:
anime_df.sample(10)

,mal_id,title1,tags
3235,51794,The Seven Deadly Sins: Four Knights of the Apo...,telecomanimationfilm action adventure fantasy ...
6641,35774,Starmyu 2nd Season Special,koukou hoshi kageki 2nd season special high sc...
9885,3812,Dream Star Button Nose,bouton d'or prinzessin erdbeer button nose top...
7247,2131,Bludgeoning Angel Dokuro-chan 2,club-to-death angel dokuro-chan 2 gaghumor gor...
1896,36754,Kakuriyo: Bed and Breakfast for Spirits,adultcast mythology gonzo drama fantasy gourme...
2611,4138,The Adventures of Scamper the Penguin,lifework soyuzmultfilm adventure drama fantasy...
8742,50422,Light Catcher,music pbanimationco.ltd. sci-fi
9346,3917,Hitozuma Cosplay Kissa 2: Hitozuma LoveLove - ...,space-x hentai
9056,3819,Nozomi In The Sun,wandering sun nathalie sasurai no taiyo music ...
5716,51941,Xing Wushen Jue 2nd Season,xing wu shen jue star martial god technique se...


In [ ]:
import nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
def stem(text):
  L = []
  for i in text.split():
    L.append(ps.stem(i))
  return " ".join(L)

In [ ]:
anime_df['tags']=anime_df['tags'].apply(stem)

<ipython-input-29-0221d66de531>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  anime_df['tags']=anime_df['tags'].apply(stem)


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 5000, stop_words = 'english')

In [ ]:
vectors = cv.fit_transform(anime_df['tags']).toarray()

In [ ]:
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
cv.get_feature_names_out()

array(['00', '000', '000000', ..., '신의', '이두나', '장금이의'], dtype=object)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity_of_eachother = cosine_similarity(vectors)

In [ ]:
def recommend(anime):
  index_anime = anime_df[anime_df['title1']==anime].index[0]
  distance = similarity_of_eachother[index_anime]

  anime_lst = sorted(list(enumerate(distance)),reverse=True, key= lambda x : x[1])[1:11]

  for i in anime_lst:
    print(anime_df.iloc[i[0]]['title1'])

In [ ]:
recommend("How a Realist Hero Rebuilt the Kingdom")

How a Realist Hero Rebuilt the Kingdom
Seirei Gensouki: Spirit Chronicles
Combatants Will Be Dispatched!
The Twelve Kingdoms
Seirei Gensouki: Spirit Chronicles Season 2
Kingdom: Season 2
Saga of Tanya the Evil
The Familiar of Zero
Anti-Magic Academy: The 35th Test Platoon
Tsukimichi -Moonlit Fantasy- Season 2
